# Classifier

In [ ]:
import random
import numpy as np
import numpy as npl
from PLS import PLS
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import matthews_corrcoef # average == 'macro'.
from sklearn.metrics import roc_auc_score # multiclas 'ovo' average == 'macro'.
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.mixture import GaussianMixture
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

import os
from sklearn.metrics import auc, precision_recall_curve, roc_auc_score
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# Step 1: List of specific CSV files to use from the directory
train_files = [
    'Augmentation/R0.csv',  # Replace with the actual filename
    'Augmentation/R25.csv',  # Replace with the actual filename
    'Augmentation/R50.csv',  # Replace with the actual filename
    'Augmentation/R75.csv',  # Replace with the actual filename
    'Augmentation/R100.csv'   # Replace with the actual filename
]

test_file_path = 'Test.csv'

# Load test data
test_df = pd.read_csv(test_file_path)
X_test = test_df.iloc[:, 1:-1].values
y_test = test_df.iloc[:, -1].values

# Apply StandardScaler to the test data
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)  # Fit and transform the test set

# Custom cross-validation function
def custom_cv(y, nr_fold):
    skf = StratifiedKFold(n_splits=nr_fold, shuffle=True, random_state=0)
    return skf.split(np.zeros(len(y)), y)

# Define cross-validation function
def cv(clf, X, y, nr_fold):
    ix = np.arange(len(y))  # Generate index array
    allACC, allSENS, allSPEC, allMCC, allROC_AUC, allBACC, allAUC_PR = [], [], [], [], [], [], []
    
    for j in range(nr_fold):
        train_ix = ((ix % nr_fold) != j)
        test_ix = ((ix % nr_fold) == j)
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        
        # Apply StandardScaler to training and testing data within each fold
        scaler = StandardScaler()
        train_X = scaler.fit_transform(train_X)
        test_X = scaler.transform(test_X)

        clf.fit(train_X, train_y)
        p = clf.predict(test_X)
        pr = clf.predict_proba(test_X)[:, 1]

        TP, FP, TN, FN = 0.00001, 0.00001, 0.00001, 0.00001
        for i in range(len(test_y)):
            if test_y[i] == 1 and p[i] == 1:
                TP += 1
            elif test_y[i] == 1 and p[i] == 0:
                FN += 1
            elif test_y[i] == 0 and p[i] == 1:
                FP += 1
            elif test_y[i] == 0 and p[i] == 0:
                TN += 1

        ACC = (TP + TN) / (TP + FP + TN + FN)
        SENS = TP / (TP + FN)
        SPEC = TN / (TN + FP)
        MCC = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) if (TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) != 0 else 0
        ROC_AUC = roc_auc_score(test_y, pr)  # Renamed from 'AUC' to 'ROC_AUC'
        F1 = TP / (TP + (0.5 * (FP + FN)))

        precision, recall, _ = precision_recall_curve(test_y, pr)
        AUC_PR = auc(recall, precision)

        allACC.append(ACC)
        allSENS.append(SENS)
        allSPEC.append(SPEC)
        allMCC.append(MCC)
        allROC_AUC.append(ROC_AUC)
        allBACC.append(F1)
        allAUC_PR.append(AUC_PR)

    return np.mean(allACC), np.mean(allSENS), np.mean(allSPEC), np.mean(allMCC), np.mean(allROC_AUC), np.mean(allBACC), np.mean(allAUC_PR)

# Define test function
def test(clf, X, y, Xt, yt):
    clf.fit(X, y)
    p = clf.predict(Xt)
    pr = clf.predict_proba(Xt)[:, 1]

    TP, FP, TN, FN = 0.00001, 0.00001, 0.00001, 0.00001
    for i in range(len(yt)):
        if yt[i] == 1 and p[i] == 1:
            TP += 1
        elif yt[i] == 1 and p[i] == 0:
            FN += 1
        elif yt[i] == 0 and p[i] == 1:
            FP += 1
        elif yt[i] == 0 and p[i] == 0:
            TN += 1

    ACC = (TP + TN) / (TP + FP + TN + FN)
    SENS = TP / (TP + FN)
    SPEC = TN / (TN + FP)
    MCC = ((TP * TN) - (FP * FN)) / np.sqrt((TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)) if (TP + FP) * (TP + FN) * (TN + FP) * (TN + FN) != 0 else 0
    ROC_AUC = roc_auc_score(yt, pr)  # Renamed from 'AUC' to 'ROC_AUC'
    F1 = TP / (TP + (0.5 * (FP + FN)))

    precision, recall, _ = precision_recall_curve(yt, pr)
    AUC_PR = auc(recall, precision)

    return ACC, SENS, SPEC, MCC, ROC_AUC, F1, AUC_PR, TP, TN, FP, FN

# Writing to CSV
output_file_path = "Try_full_1000.csv"
with open(output_file_path, "a") as file:
    file.write("Prob Feature, acc, sens, spec, mcc, roc, f1, auc_pr, tp, tn, fp, fn, best param, acc, sens, spec, mcc, roc, f1, auc_pr, tp, tn, fp, fn\n")

    # Loop through the specified files
    for filename in train_files:
        train_file_path = os.path.join('Train', filename)  # Assuming 'Train' is the folder path
        train_df = pd.read_csv(train_file_path)
        X_train = train_df.iloc[:, 1:-1].values
        y_train = train_df.iloc[:, -1].values

        # Apply StandardScaler to the training data
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        
        # Use the filename (without the extension) as the feature name
        feat = os.path.splitext(filename)[0]

        classifiers = [
        ("ET", ExtraTreesClassifier(random_state=0), {'n_estimators': [20, 50, 100, 200]}, 'n_estimators'),

    ]

        for clf_name, clf, param_grid, param_name in classifiers:
            clf1 = GridSearchCV(clf, param_grid, scoring='roc_auc', cv=custom_cv(y_train, 3), n_jobs=-1)
            clf1.fit(X_train, y_train)
            best_param = clf1.best_params_[param_name]

            clf = clf1.best_estimator_
            acc, sens, spec, mcc, roc_auc, f1, auc_pr = cv(clf, X_train, y_train, 3)
            acc1, sens1, spec1, mcc1, roc_auc1, f11, auc_pr1, TP, TN, FP, FN = test(clf, X_train, y_train, X_test, y_test)
            file.write(f"{feat}_{clf_name},{acc},{sens},{spec},{mcc},{roc_auc},{f1},{auc_pr},{best_param},{acc1},{sens1},{spec1},{mcc1},{roc_auc1},{f11},{auc_pr1},{TP},{TN},{FP},{FN}\n")
file.close()